In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Veriyi oku
data = pd.read_csv("air_pollution.csv")

# Tarih ve adres sütunlarını kaldır
# 'Measurement date' ve 'Address' sütunlarını kaldırıyoruz çünkü bu sütunlar modelin eğitiminde kullanılmayacak.
data = data.drop(columns=['Measurement date', 'Address'])

# NaN değerleri kontrol et ve temizle
# Eksik (NaN) değerleri temizliyoruz, çünkü kümeleme algoritmaları eksik verilerle çalışamaz.
data = data.dropna()

# Özelliklerin normalize edilmesi
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Özelliklerin ölçeklendirilmesi, farklı özelliklerin birbirleriyle karşılaştırılabilir olmasını sağlar.
# KMeans algoritması, uzaklık temelli bir algoritmadır ve bu yüzden özelliklerin ölçeklenmesi gereklidir.
data_normalized = scaler.fit_transform(data)

# Silhouette skoru ile uygun küme sayısını belirleme
silhouettes = []
ks = list(range(2, 12))  # 2 ile 11 arası küme sayıları
# 2'den 11'e kadar olan küme sayıları için KMeans algoritmasını uygulayıp, her bir küme sayısı için Silhouette skorunu hesaplıyoruz.
for n_cluster in ks:
    kmeans = KMeans(n_clusters=n_cluster, random_state=42).fit(data_normalized)
    label = kmeans.labels_
    # Silhouette skoru, kümelerin ne kadar iyi ayrıldığını ölçen bir metriğidir. Yüksek değerler, kümelerin daha iyi ayrıldığını gösterir.
    sil_coeff = silhouette_score(data_normalized, label, metric='euclidean')
    print(f"For n_clusters={n_cluster}, The Silhouette Coefficient is {sil_coeff}")
    silhouettes.append(sil_coeff)

# Silhouette skorlarını görselleştirme
plt.figure(figsize=(12, 8))
plt.subplot(211)
# Silhouette skorlarını ve küme sayıları arasındaki ilişkiyi görselleştiriyoruz.
plt.scatter(ks, silhouettes, marker='x', c='r')
plt.plot(ks, silhouettes)
plt.xlabel('k')
plt.ylabel('Silhouette score')
plt.title('Silhouette Score vs Number of Clusters')
plt.show()

# En iyi k sayısını seç
# En yüksek Silhouette skoruna sahip küme sayısını seçiyoruz. Bu, en uygun küme sayısını belirler.
best_k = ks[silhouettes.index(max(silhouettes))]
print(f"The best number of clusters is {best_k}")

# En iyi k sayısı ile K-Means uygulama
model = KMeans(n_clusters=best_k, random_state=42)
# En iyi küme sayısını kullanarak KMeans algoritmasını tekrar uyguluyoruz ve her veriyi bir kümeye atıyoruz.
labels = model.fit_predict(data_normalized)

# Kümeleri DataFrame'e ekle
data['Cluster'] = labels
# Veriye küme etiketlerini ekliyoruz. Böylece her bir veri noktasının hangi kümeye ait olduğunu belirleyebiliyoruz.

# Kümeler içerisindeki ortalama değerleri hesaplama
print("Average values for each cluster:")
# Her bir küme için ortalama özellik değerlerini hesaplıyoruz. Bu, kümelerin karakteristiklerini anlamamıza yardımcı olur.
for cluster_num in range(best_k):
    cluster_data = data[data['Cluster'] == cluster_num]
    print(f"Cluster {cluster_num}:")
    print(f"  Mean PM2.5: {cluster_data['PM2.5'].mean()}")
    print(f"  Mean PM10: {cluster_data['PM10'].mean()}")
    print(f"  Mean SO2: {cluster_data['SO2'].mean()}")
    print(f"  Mean NO2: {cluster_data['NO2'].mean()}")
    print(f"  Mean O3: {cluster_data['O3'].mean()}")
    print(f"  Mean CO: {cluster_data['CO'].mean()}")
    print()

# Kümeleri görselleştirme (Lat ve Long kullanarak)
plt.figure(figsize=(12, 8))
# Kümeleri görselleştiriyoruz. Bu grafikte, her bir veriyi Latitude ve Longitude'ye göre renklendiriyoruz.
plt.scatter(data['Latitude'], data['Longitude'], c=data['Cluster'], cmap='viridis', marker='o')
plt.title('Clusters based on Latitude and Longitude')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.colorbar(label='Cluster')
plt.show()
